In [1]:
import cvxpy as cvx
import numpy as np
import matplotlib.pyplot as plt
import scipy
import itertools
from scipy.sparse import bsr_matrix, csc_matrix, diags, eye

# CVXPY

In [96]:
t = cvx.Variable()

In [99]:
objective = cvx.Minimize(-t)
constraints = [A - t * np.eye(n) >> 0]
problem = cvx.Problem(objective, constraints)
optimal_value = problem.solve()

In [100]:
optimal_value

-0.0010011194220271543

In [101]:
W[0]

0.0014676378983795586

# Vectorized Implementation

In [ ]:
def form_vectorized_matrix(A_matrices):
    V_Z = form_vectorized_Z()
    V_A = form_vectorized_A(A_matrices)
    # Concat V_A to V_Z

In [174]:
def form_vectorized_Z():
    for i in range(num_blocks):
        for l in range(b ** 2):
            j = l // b
            k = l % b
            row_index = (i * (b - 1) + j) * n + (i) * (b - 1) + k
            column_index = l + i  * b ** 2
            print('Inserting 1 at ({0}, {1})'.format(row_index, column_index))
            V[row_index, column_index] = 1
    return V

In [171]:
def form_vectorized_A(A_matrices):
    # Parameter: A_matrices is a list of A_1, A_2, ..., A_k.
    #   A_matrices does NOT include A_0
    V = np.zeros(shape=(n ** 2, len(A_matrices)))
    for index in range(len(A_matrices)):
        V[:, index] = A_matrices[index].flatten()
    return V

# Utility Functions

In [772]:
def form_laplacian(n):
    A_0_sparse = diags(diagonals=[-1.0, 2.0, -1.0], offsets = [-1, 0, 1], shape=(n, n))
    return A_0_sparse.tocsc()

# Test Implementation

In [773]:
def run_alternating_projections():
    pass

In [774]:
def projection_strong_duality(A_0_sparse, A_0_nz, 
                              A_sparse_list, A_nz_list, 
                              c, 
                              x_primal_var, z_primal_var_list, 
                              lambda_dual_var,
                              sigma_dual_var_list,
                              pinv_C_cache):
    # Project y = (x, lambda) onto c'x + d'eta - tr(Lambda * A_0) = 0
    # Equivalently, projecting y onto Cy = d, where C = [c', -vec(A_0.nz)], d = 0
    C = np.hstack((c, -A_0_sparse_vectorized))
    d = np.zeros(C.shape[0])
    y = np.vstack((x_primal_var[:,np.newaxis], lambda_dual_var[A_0_nz].T))
    
    if not 'projection_strong_duality' in pinv_C_cache:
        pinv_C_cache['projection_strong_duality'] = scipy.linalg.pinv(C)
    pinv_C = pinv_C_cache['projection_strong_duality']
    
    y_prime = pinv_C.dot(d)[:, np.newaxis] + (scipy.sparse.eye(pinv_C.shape[0]) - pinv_C.dot(C)).dot(y)
    x_primal_var[:] = y_prime[0:len(x_primal_var)]
    
    lambda_dual_var[A_0_nz] = y_prime[len(x_primal_var):].T

In [775]:
def projection_dual_feasibility_equality_1(A_0_sparse, A_0_nz, 
                                          A_sparse_list, A_nz_list, 
                                          c, 
                                          x_primal_var, z_primal_var_list, 
                                          lambda_dual_var,
                                          sigma_dual_var_list,
                                          pinv_C_cache):
    # For each i, project c, lambda such that c[i] + tr(Lambda * A_i) = 0
    # Equivalently, project onto Cy = d, where C = [vec(A_i.nz)'] and y = Lambda[A_i.nz], d = -c[i]
    for i in range(len(A_sparse_list)):
        C = A_sparse_vectorized_list[i]
        d = np.array([-c[i]])
        y = lambda_dual_var[A_nz_list[i]]
        
        if not ('projection_dual_feasibility_equality_1', i) in pinv_C_cache:
            pinv_C_cache[('projection_dual_feasibility_equality_1', i)] = scipy.linalg.pinv(C[np.newaxis, :])
        pinv_C = pinv_C_cache[('projection_dual_feasibility_equality_1', i)]
        
        y_prime = pinv_C.dot(d) + (scipy.sparse.eye(pinv_C.shape[0]) - pinv_C.dot(C[np.newaxis,:])).dot(y.T)
        lambda_dual_var[A_nz_list[i]] = y_prime.T

In [776]:
def projection_dual_feasibility_equality_2(A_0_sparse, A_0_nz, 
                                          A_sparse_list, A_nz_list, 
                                          c, 
                                          x_primal_var, z_primal_var_list, 
                                          lambda_dual_var,
                                          sigma_dual_var_list,
                                          pinv_C_cache):
    for i in range(num_blocks):
        C = np.hstack((np.eye(b**2), np.eye(b**2)))
        d = np.zeros(b**2)
        diag_idx = (b-1)*i
        y = np.vstack((sigma_dual_var_list[i].flatten()[:,np.newaxis],
                       lambda_dual_var[diag_idx:diag_idx+b, diag_idx:diag_idx+b].todense().flatten().T))
        
        if not ('projection_dual_feasibility_equality_2', i) in pinv_C_cache:
            pinv_C_cache[('projection_dual_feasibility_equality_2', i)] = scipy.linalg.pinv(C)
        pinv_C = pinv_C_cache[('projection_dual_feasibility_equality_2', i)]
        
        y_prime = pinv_C.dot(d)[:, np.newaxis] + (scipy.sparse.eye(pinv_C.shape[0]) - pinv_C.dot(C)).dot(y)
        sigma_dual_var_list[i][:,:] = y_prime[:b**2].reshape(b,b)
        lambda_dual_var[diag_idx:diag_idx+b, diag_idx:diag_idx+b] = y_prime[b**2:].reshape(b,b).T
        
        check = sigma_dual_var_list[i] + lambda_dual_var[diag_idx:diag_idx + b, diag_idx:diag_idx + b]
        if not np.allclose(check, 0):
            raise Exception('Dual feasibility equality 2 projection is inconsistent')

In [777]:
def project_psd_cone(dense_A):
    W, V = scipy.linalg.eigh(dense_A)
    
    dense_A[:,:] = V @ np.diag(np.maximum(0, W)) @ V.T

In [778]:
def projection_primal_feasibility_equality(A_0_sparse, A_0_nz, 
                                          A_sparse_list, A_nz_list, 
                                          c, 
                                          x_primal_var, z_primal_var_list, 
                                          lambda_dual_var,
                                          sigma_dual_var_list,
                                          pinv_C_cache):

    cols_z = scipy.sparse.csr_matrix((len(vector_to_matrix), b**2 * num_blocks))
    rows_to_Z_idx = []

    # First map the Z_i matrices
    for curr_block_idx in range(num_blocks):
        for i, j in itertools.product(range(b), range(b)):
            ni, nj = curr_block_idx*(b-1) + i, curr_block_idx*(b-1) + j
            # Generate the basis vectors
            cols_z[matrix_to_vector[(ni, nj)], curr_block_idx*(b**2) + i*b + j] = 1

    cols_A = scipy.sparse.csr_matrix((len(vector_to_matrix), len(A_sparse_list)))
    for i in range(len(A_sparse_list)):
        cols_A[:, i] = A_sparse_list[i][sparsity_pattern_rows, sparsity_pattern_cols][:,np.newaxis]

    z_vector = np.zeros(b**2 * num_blocks)
    for block_idx in range(num_blocks):
        for i, j in itertools.product(range(b), range(b)):
            z_vector[block_idx * (b ** 2) + (i * b) + j ] = (z_primal_var_list[block_idx])[i, j]

    y = np.r_[z_vector, x_primal_var]
    d = -A_0_sparse[sparsity_pattern_rows, sparsity_pattern_cols].T
    C = scipy.sparse.hstack((-cols_z, cols_A)).todense()
    
    if not 'projection_primal_feasibility_equality' in pinv_C_cache:
        pinv_C_cache['projection_primal_feasibility_equality'] = scipy.linalg.pinv(C)
    pinv_C = pinv_C_cache['projection_primal_feasibility_equality']
    
    y_prime = pinv_C.dot(d) + (scipy.sparse.eye(pinv_C.shape[0]) - pinv_C.dot(C)).dot(y[:,np.newaxis])

    for block_idx in range(num_blocks):
        for i, j in itertools.product(range(b), range(b)):
            z_primal_var_list[block_idx][i, j] = y_prime[block_idx * (b ** 2) + (i * b) + j ]

    x_primal_var[:] = y_prime[-len(x_primal_var):]

In [779]:
def alternating_projections_iteration(A_0_sparse, A_0_nz, 
                                      A_sparse_list, A_nz_list, 
                                      c, 
                                      x_primal_var, z_primal_var_list, 
                                      lambda_dual_var,
                                      sigma_dual_var_list, 
                                      pinv_C_cache,
                                      run_checks = False):
    
    projection_strong_duality(A_0_sparse, A_0_nz, 
                              A_sparse_list, A_nz_list, 
                              c, 
                              x_primal_var, z_primal_var_list, 
                              lambda_dual_var,
                              sigma_dual_var_list,
                              pinv_C_cache)

    if run_checks:
        check = np.dot(c, x_primal_var) - (A_0_sparse @ lambda_dual_var).diagonal().sum()
        if not np.allclose(check, 0):
            raise Exception('Strong duality projection is inconsistent')

    projection_dual_feasibility_equality_1(A_0_sparse, A_0_nz, 
                                              A_sparse_list, A_nz_list, 
                                              c, 
                                              x_primal_var, z_primal_var_list, 
                                              lambda_dual_var,
                                              sigma_dual_var_list,
                                              pinv_C_cache)

    if run_checks:
        for i in range(len(A_sparse_list)):
            check = c[i] + (lambda_dual_var @ A_sparse_list[i]).diagonal().sum()
            if not np.allclose(check, 0):
                raise Exception('Dual feasibility equality 1 projection is inconsistent')
        
    for _ in range(1):
        projection_dual_feasibility_equality_2(A_0_sparse, A_0_nz, 
                                              A_sparse_list, A_nz_list, 
                                              c, 
                                              x_primal_var, z_primal_var_list, 
                                              lambda_dual_var,
                                              sigma_dual_var_list,
                                              pinv_C_cache)
    
    
    projection_primal_feasibility_equality(A_0_sparse, A_0_nz, 
                                          A_sparse_list, A_nz_list, 
                                          c, 
                                          x_primal_var, z_primal_var_list, 
                                          lambda_dual_var,
                                          sigma_dual_var_list,
                                          pinv_C_cache)
    
    if run_checks:
        Lx = A_0_sparse
        for i in range(len(A_sparse_list)):
            Lx += A_sparse_list[i] * x_primal_var[i]

        Z = np.zeros((n, n))
        for block_idx in range(num_blocks):
            diag_idx = block_idx*(b-1)
            Z[diag_idx:diag_idx + b, diag_idx:diag_idx + b] += z_primal_var_list[block_idx]

        check = Lx - Z
        if not np.allclose(check, 0):
            print(check)
            raise Exception('Primal feasibility equality projection is inconsistent')

        
    for i in range(len(sigma_dual_var_list)):
        project_psd_cone(sigma_dual_var_list[i])
        project_psd_cone(z_primal_var_list[i])
    

# Test Implementation

In [786]:
num_A = 1
num_blocks = 10
b = 3
n = b + (num_blocks - 1) * (b - 1)

In [787]:
# Project onto L(x) = \sum E_i Z_i E_i'
# generate indices of the nonzero entires for block sparsity pattern
matrix_to_vector= {} # Map (i, j) to k
vector_to_matrix_set = set() 
for block_index in range(num_blocks):
    top_left = ((b - 1) * block_index, (b - 1) * block_index)
    for (di, dj) in itertools.product(range(b), range(b)):
        vector_to_matrix_set.add((top_left[0] + di, top_left[1] + dj))
vector_to_matrix = list(vector_to_matrix_set)
sparsity_pattern_rows = np.array([op[0] for op in vector_to_matrix])
sparsity_pattern_cols = np.array([op[1] for op in vector_to_matrix])
matrix_to_vector = {p:i for i, p in enumerate(vector_to_matrix)}

In [788]:
A_0_sparse = form_laplacian(n)
A_0_nz = A_0_sparse.nonzero()
A_0_sparse_vectorized = A_0_sparse[A_0_nz]

A_sparse_list = [-np.eye(n)]
A_nz_list = [A_sparse_list[i].nonzero() for i in range(len(A_sparse_list))]
A_sparse_vectorized_list = [A_sparse_list[i][A_nz_list[i]] for i in range(len(A_sparse_list))]

c = -np.ones((1,1))
# x_primal_var = np.zeros(1)
x_primal_var = np.zeros(1)
z_primal_var_list = [np.zeros((b,b)) for _ in range(num_blocks)]
lambda_dual_var = scipy.sparse.csr_matrix((n,n))
sigma_dual_var_list = [np.zeros((b,b)) for _ in range(num_blocks)]
pinv_C_cache = {}
params = [
    A_0_sparse, A_0_nz, 
    A_sparse_list, A_nz_list, 
    c, 
    x_primal_var, z_primal_var_list, 
    lambda_dual_var,
    sigma_dual_var_list,
    pinv_C_cache
]

In [789]:
for iteration in range(2000):
    if iteration % 100 == 0:
        print(x_primal_var)
    alternating_projections_iteration(*params, run_checks=True)

[0.]


/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


[0.00584627]
[0.00616038]
[0.00630331]
[0.00638706]
[0.00646565]
[0.00653995]
[0.00661069]
[0.00667845]
[0.00674365]
[0.00680665]
[0.00686769]
[0.00692699]
[0.00698472]
[0.00704102]
[0.00709602]
[0.00714982]
[0.0072025]
[0.00725415]
[0.00730483]


In [790]:
np.linalg.eigvalsh(A_0_sparse.todense()).min()

0.020357116238134833